# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846838065676                   -0.70    4.0
  2   -7.852316675546       -2.26       -1.53    1.0
  3   -7.852646079444       -3.48       -2.52    3.2
  4   -7.852646676723       -6.22       -3.34    2.2
  5   -7.852646685861       -8.04       -4.77    1.0
  6   -7.852646686726       -9.06       -5.19    4.0
  7   -7.852646686729      -11.45       -5.88    1.8
  8   -7.852646686730      -12.44       -7.27    2.0
  9   -7.852646686730      -15.05       -7.61    2.5
 10   -7.852646686730   +  -14.75       -8.57    2.2
 11   -7.852646686730   +  -14.75       -9.95    2.2
 12   -7.852646686730   +    -Inf      -10.56    2.0
 13   -7.852646686730      -14.35      -11.38    2.5
 14   -7.852646686730   +  -14.35      -11.52    1.0
 15   -7.852646686730      -14.75      -11.44    1.0
 16   -7.852646686730   +    -Inf      -11.28    1.0
 17   -7.852646686730   +    -Inf      -11.20 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846819333975                   -0.70           4.2
  2   -7.852524610368       -2.24       -1.62   0.80    5.0
  3   -7.852610796720       -4.06       -2.79   0.80    5.2
  4   -7.852646566376       -4.45       -3.57   0.80    7.0
  5   -7.852646683295       -6.93       -4.48   0.80    8.8
  6   -7.852646686668       -8.47       -5.08   0.80   10.5
  7   -7.852646686725      -10.25       -6.23   0.80   11.5
  8   -7.852646686730      -11.29       -7.18   0.80   13.5
  9   -7.852646686730      -13.64       -7.69   0.80   15.5
 10   -7.852646686730   +  -14.57       -9.62   0.80   16.8
 11   -7.852646686730      -14.75       -9.58   0.80   19.0
 12   -7.852646686730      -14.75      -10.87   0.80   20.0
 13   -7.852646686730   +  -14.75      -11.80   0.80   21.8
 14   -7.852646686730      -14.45      -12.38   0.80   23.8


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.408833e+01     3.448768e+00
 * time: 0.43297719955444336
     1     1.421088e+00     1.737928e+00
 * time: 0.6463110446929932
     2    -1.444889e+00     2.089293e+00
 * time: 0.6720960140228271
     3    -3.578585e+00     1.729548e+00
 * time: 0.7091691493988037
     4    -4.900931e+00     1.606836e+00
 * time: 0.7464561462402344
     5    -6.621558e+00     9.877220e-01
 * time: 0.7836480140686035
     6    -7.301899e+00     3.221510e-01
 * time: 0.8205900192260742
     7    -7.527342e+00     1.588221e-01
 * time: 0.8465399742126465
     8    -7.635508e+00     1.653076e-01
 * time: 0.8721551895141602
     9    -7.715895e+00     1.199263e-01
 * time: 0.8981020450592041
    10    -7.779716e+00     9.983116e-02
 * time: 0.9233930110931396
    11    -7.812171e+00     8.648838e-02
 * time: 0.9482061862945557
    12    -7.833440e+00     5.640855e-02
 * time: 0.9730551242828369
    13    -7.841181e+00     5.315456e-02
 * time: 0.998897075

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846697801839                   -0.70    4.0
  2   -7.852300227419       -2.25       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686618                   -2.54
  2   -7.852646686730       -9.95       -5.66
  3   -7.852646686730   +  -14.75      -11.43
  4   -7.852646686730   +    -Inf      -15.78


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.873220130649322e-13
|ρ_newton - ρ_scfv| = 3.845971959160428e-13
|ρ_newton - ρ_dm|   = 2.453290510979936e-9
